In [16]:
import os

In [17]:
%pwd

'y:\\Documents\\ML_AI\\Smartphone price prediction\\PriceSpeculator\\research'

In [18]:
cd ..

y:\Documents\ML_AI\Smartphone price prediction\PriceSpeculator


In [19]:
%pwd

'y:\\Documents\\ML_AI\\Smartphone price prediction\\PriceSpeculator'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str

In [10]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        #Here you can add params also incase you are doing hyperparameter tuining.
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path = config.test_data_path,
            model_name= config.model_name,
            target_column= schema.name 
        )
        
        return model_trainer_config

In [12]:
import pandas as pd
import os
from mlProject import logging
from sklearn.ensemble import RandomForestRegressor
import joblib
from sklearn.preprocessing import StandardScaler

In [13]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]
        
        scaler = StandardScaler()
        train_scaled = scaler.fit_transform(train_x)
        test_scaled = scaler.transform(test_x)
        
        lr = RandomForestRegressor()
        lr.fit(train_scaled,train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [20]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-03-21 14:46:08,372: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-21 14:46:08,380: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-21 14:46:08,385: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-21 14:46:08,389: INFO: common: Created directory at: artifacts]


[2025-03-21 14:46:08,391: INFO: common: Created directory at: artifacts/model_trainer]
